# Loading Pre-Processed Dataset

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
model_dir = '/content/drive/MyDrive/Implementation/Model_BinaryLoss_Cosine/'

Loading the numpy files created from the orginal notebook (Contrastive Loss with Cosine Similarity) to avoid redundancy.

In [ ]:
import numpy as np
import os

# Load test data from Google Drive
test_data_path = '/content/drive/MyDrive/Implementation/dataset_preprocessed'

# Load train data from the specified path
X_train_1 = np.load(os.path.join(test_data_path, 'X_train_1.npy'))
X_train_2 = np.load(os.path.join(test_data_path, 'X_train_2.npy'))
train_labels = np.load(os.path.join(test_data_path, 'train_labels.npy'))

# Load validation data from the specified path
X_val_1 = np.load(os.path.join(test_data_path, 'X_val_1.npy'))
X_val_2 = np.load(os.path.join(test_data_path, 'X_val_2.npy'))
val_labels = np.load(os.path.join(test_data_path, 'val_labels.npy'))

# LSTM MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

''' A Contrastive Loss funtion provided by Keras (https://keras.io/examples/vision/siamese_contrastive/#define-the-contrastive-loss)'''

def loss(margin=1):
    """Provides 'contrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'contrastive_loss' function with data ('margin') attached.
    """

    # Contrastive loss = mean( (1-true_value) * square(prediction) + true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the contrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing contrastive loss as floating point value.
        """

        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - (y_pred), 0))
        return tf.reduce_mean((1 - y_true) * square_pred + (y_true) * margin_square)

    return contrastive_loss

In [ ]:
input = tf.keras.layers.Input((200,6))
x = tf.keras.layers.LSTM(512, return_sequences=True)(input)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.LSTM(256, return_sequences=True)(x)
x = tf.keras.layers.LSTM(128)(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)

embedding_network = tf.keras.Model(input, x)

input_1 = tf.keras.layers.Input((200,6))
input_2 = tf.keras.layers.Input((200,6))

tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = tf.keras.layers.Dot(axes=1, normalize = True)([tower_1, tower_2])

output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(merge_layer)

keras_siamese = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [ ]:
keras_siamese.compile(loss=loss(margin=1), optimizer="RMSprop", metrics=["accuracy"])
keras_siamese.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 200, 6)]             0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 200, 6)]             0         []                            
                                                                                                  
 model_2 (Functional)        (None, 64)                   2072256   ['input_5[0][0]',             
                                                                     'input_6[0][0]']             
                                                                                                  
 dot (Dot)                   (None, 1)                    0         ['model_2[0][0]',       

## Defining the training checkpoints

In [ ]:
from datetime import datetime
import pytz

# Specify Sri Lanka timezone
sri_lanka_timezone = pytz.timezone('Asia/Colombo')

# Get the current time in UTC
utc_now = datetime.utcnow()

# Convert the UTC time to Sri Lanka's local time
local_time = utc_now.replace(tzinfo=pytz.utc).astimezone(sri_lanka_timezone)

# Format the local time as a string
formatted_local_time = local_time.strftime("%Y-%m-%d_%H:%M")

print("Formatted Local Time in Sri Lanka:", formatted_local_time)

Formatted Local Time in Sri Lanka: 2024-03-31_12:43


In [ ]:
import os

checkpoint_dir = os.path.join(model_dir, 'training_checkpoints/')

# Define checkpoint filepath
checkpoint_path = os.path.join(checkpoint_dir, f'{formatted_local_time}_cp.ckpt')

print("Checkpoint Path:", checkpoint_path)

Checkpoint Path: /content/drive/MyDrive/Implementation/Model_BinaryLoss_Cosine/training_checkpoints/2024-03-31_12:43_cp.ckpt


In [ ]:
# Tensorboard logs dir
log_dir = os.path.join(model_dir, 'tensorboard_logs/')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

# Define the ModelCheckpoint callback to save weights
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      save_weights_only=True,
                                      save_best_only=True,
                                      verbose=1)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Monitor validation loss, stop after 3 epochs without improvement
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# Train the model from a saved checkpoint
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

print("Last Checkpoint: ", latest_checkpoint)

if latest_checkpoint:  # If a checkpoint exists
    keras_siamese.load_weights(latest_checkpoint)
    print('Loaded weights from checkpoint:', latest_checkpoint)
else:
    print('No checkpoint found. Starting training from scratch.')

Last Checkpoint:  None
No checkpoint found. Starting training from scratch.


In [ ]:
history = keras_siamese.fit(
    [X_train_1, X_train_2],  # Training pairs
    train_labels,  # Training labels
    epochs=10,  # Number of epochs
    batch_size=64,  # Batch size
    validation_data=([X_val_1, X_val_2], val_labels),  # Validation data
    callbacks = [checkpoint_callback, tensorboard_callback, early_stopping]
)

Epoch 1/10
149/149 [==============================] - ETA: 0s - loss: 0.2515 - accuracy: 0.4918 
Epoch 1: val_loss improved from 0.26324 to 0.25716, saving model to /content/drive/MyDrive/Implementation/Model_BinaryLoss_Cosine/training_checkpoints/2024-03-31_12:43_cp.ckpt
149/149 [==============================] - 2045s 14s/step - loss: 0.2515 - accuracy: 0.4918 - val_loss: 0.2572 - val_accuracy: 0.4975
Epoch 2/10
149/149 [==============================] - ETA: 0s - loss: 0.2516 - accuracy: 0.4809 
Epoch 2: val_loss improved from 0.25716 to 0.25639, saving model to /content/drive/MyDrive/Implementation/Model_BinaryLoss_Cosine/training_checkpoints/2024-03-31_12:43_cp.ckpt
149/149 [==============================] - 2071s 14s/step - loss: 0.2516 - accuracy: 0.4809 - val_loss: 0.2564 - val_accuracy: 0.4966
Epoch 3/10
149/149 [==============================] - ETA: 0s - loss: 0.2512 - accuracy: 0.4924 
Epoch 3: val_loss improved from 0.25639 to 0.25453, saving model to /content/drive/MyDriv

In [ ]:
%tensorboard --logdir={log_dir}

# Save and Export Model

In [ ]:
# Save the entire model to a HDF5 file (including architecture, weights, and training configuration)
keras_siamese.save('siamese_model.tflite')

In [ ]:
# Compress the folder
!zip -r /content/model_03_18.zip /content/siamese_model.tflite

# Generate a download link
from google.colab import files
files.download("/content/model_03_18.zip")